In [84]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy import stats
import early_warning_detection_functions
import matplotlib.pyplot as plt
from statsmodels.stats.contingency_tables import mcnemar


# Read data

In [2]:
df_aps = pd.read_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_phc_only_imed.parquet')

df_otc = pd.read_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_otc_only_imed.parquet')

df_aih = pd.read_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_aih_imed2.parquet')



In [3]:
dta1 = df_aps[[ 'co_imed', 'Nome_imed', 'year_week', 'year_week_ts', 'out_of_limits_above_phc_only']]
dta2 = df_otc[[ 'co_imed', 'year_week', 'out_of_limits_above_otc_only']]
dta3 = df_aih[[ 'co_imed', 'year_week', 'warning_final_aih']]


result = dta1.merge(dta2, left_on=['co_imed','year_week'], right_on=['co_imed','year_week'])

result = result.merge(dta3, left_on=['co_imed','year_week'], right_on=['co_imed','year_week'])

In [4]:
result

,co_imed,Nome_imed,year_week,year_week_ts,out_of_limits_above_phc_only,out_of_limits_above_otc_only,warning_final_aih
0,120003,Sena Madureira,2022-47,2022-11-21,0,0,0
1,120003,Sena Madureira,2022-48,2022-11-28,0,0,0
2,120003,Sena Madureira,2022-49,2022-12-05,0,0,0
3,120003,Sena Madureira,2022-50,2022-12-12,0,0,0
4,120003,Sena Madureira,2022-51,2022-12-19,0,0,0
...,...,...,...,...,...,...,...
55806,350049,São José dos Campos,2024-48,2024-11-25,1,0,0
55807,350049,São José dos Campos,2024-49,2024-12-02,1,0,0
55808,350049,São José dos Campos,2024-50,2024-12-09,1,0,0
55809,350049,São José dos Campos,2024-51,2024-12-16,1,0,0


# Construct contigency table per strata

In [72]:
def antici_sets(data_res, col_warn_s1, col_warn_s2, col_code):
    
    lst_count = []

    for code in data_res[col_code].unique():
    
        dta = data_res[data_res[col_code] == code].copy()
        
        set3 = dta[(dta[col_warn_s1] == 1) & (dta[col_warn_s2].shift(-3) == 1)].index + 3
        set2 = dta[(dta[col_warn_s1] == 1) & (dta[col_warn_s2].shift(-3) == 0) & (dta[col_warn_s2].shift(-2) == 1)].index + 2
        set1 = dta[(dta[col_warn_s1] == 1) & 
                   (dta[col_warn_s2].shift(-3) == 0) & 
                   (dta[col_warn_s2].shift(-2) == 0) & 
                   (dta[col_warn_s2].shift(-1) == 1)].index + 1
        set0 = dta[(dta[col_warn_s1] == 1) & 
                   (dta[col_warn_s2] == 1) & 
                   (dta[col_warn_s2].shift(-3) == 0) & 
                   (dta[col_warn_s2].shift(-2) == 0) & 
                   (dta[col_warn_s2].shift(-1) == 0)].index
        
        set_3_0 = set3.union(set2).union(set1).union(set0)
        
        lst_count.append([code, set_3_0, dta.warning_final_aih.sum()])
    
    return lst_count

In [73]:
lst_aps = antici_sets(result, 'out_of_limits_above_phc_only', 'warning_final_aih', 'co_imed')

In [75]:
lst_otc = antici_sets(result, 'out_of_limits_above_otc_only', 'warning_final_aih', 'co_imed')

In [106]:
lst = []

for i in range(0,len(lst_aps)):

    n_phc_yes_otc_yes = len(lst_aps[i][1].intersection(lst_otc[i][1]))

    n_phc_yes_otc_no = len(set(lst_aps[i][1]) - set(lst_otc[i][1]))

    n_phc_no_otc_yes = len(set(lst_otc[i][1]) - set(lst_aps[i][1]))

    n_phc_no_otc_no = lst_aps[i][2] - (n_phc_yes_otc_yes + n_phc_yes_otc_no + n_phc_no_otc_yes)
    
    lst.append([lst_aps[i][0], n_phc_yes_otc_yes, n_phc_yes_otc_no, n_phc_no_otc_yes, n_phc_no_otc_no])

In [107]:
df_counts = pd.DataFrame(lst,
       columns=['co_imed', 'n_phc_yes_otc_yes', 'n_phc_yes_otc_no','n_phc_no_otc_yes','n_phc_no_otc_no'])


## Format data to apply the test

In [108]:
# Read dictionary with municipalities
df_imed_key = pd.read_csv('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Data/RELATORIO_DTB_BRASIL_MUNICIPIO.csv',sep = ';')
df_pop = pd.read_csv('/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Population/table_2009_2023_etcc_eq_zero.csv')


In [109]:
df_pop = df_pop[['Cód.', 'Uf', '2022']]

df_imed_key = df_imed_key[['Nome_UF', 'UF',  'Região Geográfica Imediata',
       'Nome Região Geográfica Imediata', 
       'Município', 'Código Município Completo', 'Nome_Município']]

df_pop = df_pop.rename(columns = {'Cód.': 'co_ibge7'}) 

df_imed_key = df_imed_key.rename(columns = {'Código Município Completo': 'co_ibge7', 
                                            'Região Geográfica Imediata': 'co_imed'})

df_imed_key = df_imed_key.merge(df_pop, on='co_ibge7', how='left')

df_imed_pop = df_imed_key.groupby(['co_imed'])['2022'].sum().reset_index()

df_imed_pop = df_imed_pop.assign(pop_2022 = df_imed_pop['2022'].astype(int))
df_imed_pop = df_imed_pop[['co_imed', 'pop_2022']]


df_codes_imed = df_imed_key.groupby(['Nome_UF','UF', 'co_imed','Uf'])['2022'].sum().reset_index()



In [110]:
# Dictionary mapping Brazilian states to their regions
state_to_region = {
    "AC": "North", "AM": "North", "AP": "North", "PA": "North", "RO": "North", "RR": "North", "TO": "North",
    "AL": "Northeast", "BA": "Northeast", "CE": "Northeast", "MA": "Northeast", "PB": "Northeast",
    "PE": "Northeast", "PI": "Northeast", "RN": "Northeast", "SE": "Northeast",
    "DF": "Center-West", "GO": "Center-West", "MT": "Center-West", "MS": "Center-West",
    "ES": "Southeast", "MG": "Southeast", "RJ": "Southeast", "SP": "Southeast",
    "PR": "South", "RS": "South", "SC": "South"
}


df_counts2 = df_counts.merge(df_codes_imed, on='co_imed', how='left')

# Create a new column 'region' by mapping state codes to regions
df_counts2["region"] = df_counts2["Uf"].map(state_to_region)


In [111]:
df_counts2 = df_counts2.rename(columns = {'2022': 'pop'})

In [112]:
bins = [0, 1.177890e+05, 3.247680e+05, float('inf')]
labels = ['Small', 'Medium', 'Large']

df_counts2['pop_cat'] = pd.cut(df_counts2['pop'], bins=bins, labels=labels, right=True)

##  analysis

In [114]:
from statsmodels.stats.contingency_tables import mcnemar

# Contingency table
#           OTC Yes | OTC No
# PHC Yes     a     |   b
# PHC No      c     |   d

#[[268, 81],
#[123, 84]]

# Helper function to compute and print McNemar test
def run_mcnemar_test(label, df_subset):
    a = df_subset.n_phc_yes_otc_yes.sum()
    b = df_subset.n_phc_yes_otc_no.sum()
    c = df_subset.n_phc_no_otc_yes.sum()
    d = df_subset.n_phc_no_otc_no.sum()

    table = [[a, b],
             [c, d]]

    result = mcnemar(table, exact=True)
    print(f"{label:<15} | Statistic: {result.statistic:<5} | p-value: {result.pvalue:.4f}", table)

# National-level analysis
print(" McNemar Test Results – Nationwide")
run_mcnemar_test("Nationwide", df_counts2)

# Macro-region analysis
print("\n McNemar Test by Macro Region")
for region in sorted(df_counts2.region.unique()):
    df_region = df_counts2[df_counts2.region == region]
    run_mcnemar_test(region, df_region)

# Population size category analysis
print("\n McNemar Test by Population Size Category")
for cat in sorted(df_counts2.pop_cat.unique()):
    df_cat = df_counts2[df_counts2.pop_cat == cat]
    run_mcnemar_test(cat, df_cat)


 McNemar Test Results – Nationwide
Nationwide      | Statistic: 81.0  | p-value: 0.0040 [[268, 81], [123, 84]]

 McNemar Test by Macro Region
Center-West     | Statistic: 14.0  | p-value: 0.8555 [[39, 16], [14, 4]]
North           | Statistic: 14.0  | p-value: 0.3915 [[45, 14], [20, 14]]
Northeast       | Statistic: 17.0  | p-value: 0.0022 [[82, 17], [41, 36]]
South           | Statistic: 15.0  | p-value: 0.6076 [[37, 15], [19, 11]]
Southeast       | Statistic: 19.0  | p-value: 0.1934 [[65, 19], [29, 19]]

 McNemar Test by Population Size Category
Large           | Statistic: 17.0  | p-value: 0.4296 [[52, 17], [23, 21]]
Medium          | Statistic: 43.0  | p-value: 0.0346 [[144, 43], [66, 37]]
Small           | Statistic: 21.0  | p-value: 0.1048 [[72, 21], [34, 26]]


In [115]:
268 + 81+ 123 + 84



556

In [117]:
39+ 16 +14+ 4

73